<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/brownianbridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper to mimic C++ maxDiff
def max_diff(iter1, iter2):
    diff = 0.0
    for x, y in zip(iter1, iter2):
        diff = max(diff, abs(x - y))
    return diff

# Helper to mimic C++ maxRelDiff
def max_rel_diff(iter1, iter2):
    diff = 0.0
    for x, y in zip(iter1, iter2):
        if abs(y) > 1e-12: # Avoid division by zero or very small numbers
            diff = max(diff, abs((x - y) / y))
        elif abs(x-y) > 1e-9: # If y is zero, but x is not, it's a large relative diff
             diff = float('inf')
        # If both x and y are very small, relative diff is small or undefined, treat as 0 here if abs(x-y) is small.
    return diff


class BrownianBridgeTests(unittest.TestCase):

    def setUp(self):
        self.original_eval_date = ql.Settings.instance().evaluationDate
        # Some tests might modify this, so we save and restore.

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.original_eval_date


    def testVariates(self):
        print("Testing Brownian-bridge variates...")

        times = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 5.0]
        N = len(times)
        samples = 262143  # 2^18 - 1
        seed = 42

        # Using SobolRsg; dimension is N for the number of time steps.
        # In QL Python, SobolRsg takes dimension and seed.
        sobol_rsg = ql.SobolRsg(N, seed)
        # InverseCumulativeRsg uses the underlying SobolRsg and an InverseCumulativeNormal
        # In QL Python, InverseCumulativeRsg is a template, but often specific versions are wrapped.
        # The most common is InverseCumulativeNormal.
        generator = ql.InverseCumulativeRsgSobolGaussian(sobol_rsg) # This is a common alias

        bridge = ql.BrownianBridge(times)

        stats1 = ql.SequenceStatistics(N)
        stats2 = ql.SequenceStatistics(N)
        temp = ql.Array(N) # Use QL Array for compatibility with transform

        for _ in range(samples):
            sample_seq = generator.nextSequence() # sample_seq.value is a list/Array

            # Convert sample_seq.value to a list if it's not already suitable for iteration
            # BrownianBridge.transform expects iterators. QL Array should provide them.
            # If sample_seq.value is a QL Array, it can be used directly for iterators.
            # If it's a Python list, it's fine.
            # Ensure types match for bridge.transform.
            # Python's ql.BrownianBridge().transform() takes two ql.Array arguments.

            # The C++ transform takes iterators. Python's might take QL Arrays.
            # Let's assume sample_seq.value needs to be put into a QL Array.
            sample_array = ql.Array(list(sample_seq.value)) # Ensure it's a QL Array

            bridge.transform(sample_array, temp) # temp is modified in-place
            stats1.add(temp) # add takes a QL Array

            # For stats2, denormalize and sum along the path
            # temp now holds the transformed values from bridge.transform
            temp_path = ql.Array(N) # Create a new array for this path
            temp_path[0] = temp[0] * math.sqrt(times[0])
            for j in range(1, N):
                temp_path[j] = temp_path[j-1] + temp[j] * math.sqrt(times[j] - times[j-1])
            stats2.add(temp_path)

        # Normalized single variates (stats1)
        expected_mean_1 = [0.0] * N
        expected_covariance_1_list = [[0.0]*N for _ in range(N)]
        for i in range(N):
            expected_covariance_1_list[i][i] = 1.0
        expected_covariance_1 = ql.Matrix(expected_covariance_1_list) # Convert to QL Matrix

        # Tolerances from C++
        mean_tolerance = 1.0e-14 if ql.CompiledAs().fastMath else 1.0e-16 # Adjusted based on typical Python float precision
        cov_tolerance_1 = 2.5e-4

        mean_1 = list(stats1.mean()) # Convert QL Array to list for zip
        covariance_1 = stats1.covariance() # This is a QL Matrix

        # Max error for mean (vector)
        max_mean_error_1 = max_diff(mean_1, expected_mean_1)

        # Max error for covariance (matrix)
        # ql.Matrix elements can be accessed by [row][col]
        # We need to iterate through all elements of the matrix.
        cov_elements_1 = [covariance_1[r][c] for r in range(N) for c in range(N)]
        exp_cov_elements_1 = [expected_covariance_1[r][c] for r in range(N) for c in range(N)]
        max_cov_error_1 = max_diff(cov_elements_1, exp_cov_elements_1)


        self.assertLessEqual(max_mean_error_1, mean_tolerance,
                             msg=(f"failed to reproduce expected mean values (stats1)"
                                  f"\n    calculated: {mean_1}"
                                  f"\n    expected:   {expected_mean_1}"
                                  f"\n    max error:  {max_mean_error_1}"))

        self.assertLessEqual(max_cov_error_1, cov_tolerance_1,
                             msg=(f"failed to reproduce expected covariance (stats1)\n"
                                  # f"    calculated:\n{covariance_1}" # Printing full matrix can be verbose
                                  # f"    expected:\n{expected_covariance_1}"
                                  f"    max error:  {max_cov_error_1}"))

        # Denormalized sums along the path (stats2)
        expected_mean_2 = [0.0] * N
        expected_covariance_2_list = [[0.0]*N for _ in range(N)]
        for i in range(N):
            for j in range(i, N):
                expected_covariance_2_list[i][j] = times[i]
                expected_covariance_2_list[j][i] = times[i]
        expected_covariance_2 = ql.Matrix(expected_covariance_2_list)

        cov_tolerance_2 = 6.0e-4

        mean_2 = list(stats2.mean())
        covariance_2 = stats2.covariance()

        max_mean_error_2 = max_diff(mean_2, expected_mean_2)
        cov_elements_2 = [covariance_2[r][c] for r in range(N) for c in range(N)]
        exp_cov_elements_2 = [expected_covariance_2[r][c] for r in range(N) for c in range(N)]
        max_cov_error_2 = max_diff(cov_elements_2, exp_cov_elements_2)

        self.assertLessEqual(max_mean_error_2, mean_tolerance,
                             msg=(f"failed to reproduce expected mean values (stats2)"
                                  f"\n    calculated: {mean_2}"
                                  f"\n    expected:   {expected_mean_2}"
                                  f"\n    max error:  {max_mean_error_2}"))

        self.assertLessEqual(max_cov_error_2, cov_tolerance_2,
                             msg=(f"failed to reproduce expected covariance (stats2)\n"
                                  # f"    calculated:\n{covariance_2}"
                                  # f"    expected:\n{expected_covariance_2}"
                                  f"    max error:  {max_cov_error_2}"))


    def testPathGeneration(self):
        print("Testing Brownian-bridge path generation...")

        times = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 5.0, 7.0, 9.0, 10.0]
        grid = ql.TimeGrid(times)
        N = len(times) # N here is number of time points excluding t=0 for stats

        # PathGenerator returns paths of length grid.size().
        # The stats are on path values at times[0]...times[N-1], so N values.
        # path[0] is initial value, path[1] is value at times[0], ..., path[N] is value at times[N-1]
        # So, stats should be on N values.

        samples = 131071 # 2^17 - 1
        seed = 42
        # SobolRsg dimension is number of steps in the path, which is grid.size() - 1 or N
        # PathGenerator uses random numbers for each step. If grid has M points, there are M-1 steps.
        # Here N = len(times) is also grid.size(). So, N steps.
        # However, PathGenerator needs `dimension = process.factors() * (timeGrid.size() - 1)`
        # For 1D process, dimension = grid.size() - 1 = N - 1.
        # The C++ code uses `SobolRsg sobol(N, seed);` where N is `times.size()`.
        # This seems to imply that if grid has N points, the PathGenerator expects N random numbers per path.
        # Let's recheck PathGenerator's requirement for RNG dimension.
        # MultiPathGenerator: `dimension_ = process_->factors() * (timeGrid.size() - 1);`
        # PathGenerator (single factor): `dimension_ = (timeGrid.size() - 1);`
        # So, if `times` has N elements, `grid.size()` is N. Dimension for RNG is N-1.
        # C++ test uses `SobolRsg sobol(N, seed);` where N is `times.size()`. This might be an oversight if PathGenerator only uses N-1 of them.
        # Or, it means the BrownianBridge itself might use all N.
        # Let's stick to C++ test's Sobol dim: N = len(times).
        sobol_rsg = ql.SobolRsg(N, seed)
        gsg = ql.InverseCumulativeRsgSobolGaussian(sobol_rsg)

        today = ql.Settings.instance().evaluationDate # Use current setting
        x0_quote = ql.QuoteHandle(ql.SimpleQuote(100.0))
        r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.06, ql.Actual365Fixed()))
        q_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual365Fixed()))
        sigma_ts = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, ql.NullCalendar(), 0.20, ql.Actual365Fixed()))

        process = ql.BlackScholesMertonProcess(x0_quote, q_ts, r_ts, sigma_ts)

        # PathGenerator(process, timeGrid, generator, brownianBridge)
        generator1 = ql.PathGeneratorSobolGaussian(process, grid, gsg, False) # No Brownian Bridge
        generator2 = ql.PathGeneratorSobolGaussian(process, grid, gsg, True)  # With Brownian Bridge

        # Stats will be on N values (path[1] to path[N])
        stats1 = ql.SequenceStatistics(N)
        stats2 = ql.SequenceStatistics(N)
        temp = ql.Array(N)

        for _ in range(samples):
            path1_obj = generator1.next()
            path1 = path1_obj.value # path1 is a ql.Path object
            # Path object has `length()` and `[i]` accessors.
            # Copy values from path1[1] onwards. path1 has N+1 elements (0 to N).
            # Grid has N points (times[0] to times[N-1]). Path values correspond to these.
            # path1[0] = S0, path1[1] = S(t1), ..., path1[N] = S(tN)
            # The stats are on path values S(t_i), so N values.
            for j in range(N):
                temp[j] = path1[j+1] # path1 has N+1 elements. grid has N points. stats on N values.
            stats1.add(temp)

            path2_obj = generator2.next()
            path2 = path2_obj.value
            for j in range(N):
                temp[j] = path2[j+1]
            stats2.add(temp)

        expected_mean = list(stats1.mean())
        expected_covariance = stats1.covariance() # QL Matrix

        mean_bb = list(stats2.mean())
        covariance_bb = stats2.covariance() # QL Matrix

        mean_tolerance = 3.0e-5
        cov_tolerance = 3.0e-3 # Relative tolerance

        max_mean_error_rel = max_rel_diff(mean_bb, expected_mean)

        # Relative diff for covariance matrices
        cov_elements_bb = [covariance_bb[r][c] for r in range(N) for c in range(N)]
        exp_cov_elements = [expected_covariance[r][c] for r in range(N) for c in range(N)]
        max_cov_error_rel = max_rel_diff(cov_elements_bb, exp_cov_elements)


        self.assertLessEqual(max_mean_error_rel, mean_tolerance,
                             msg=(f"failed to reproduce expected mean values (path gen)"
                                  f"\n    calculated (BB): {ql.Array(mean_bb)}" # Print as QL Array for consistency
                                  f"\n    expected (Std):  {ql.Array(expected_mean)}"
                                  f"\n    max rel error:   {max_mean_error_rel}"))

        self.assertLessEqual(max_cov_error_rel, cov_tolerance,
                             msg=(f"failed to reproduce expected covariance (path gen)\n"
                                  # f"    calculated (BB):\n{covariance_bb}"
                                  # f"    expected (Std):\n{expected_covariance}"
                                  f"    max rel error:  {max_cov_error_rel}"))


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)